# Building Training Data for FMC Models

The purpose of this notebook is to combine the weather data from OK Mesonet, and from Van der Kamp when necessary, with the field observations from Carlson into datasets for training and evaluating models of FMC for various fuel classes.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("src")
from utils import read_yml

In [ ]:
df1    = pd.read_excel("data/processed_data/ok_1h.xlsx")
df10   = pd.read_excel("data/processed_data/ok_10h.xlsx")
df100  = pd.read_excel("data/processed_data/ok_100h.xlsx")
df1000 = pd.read_excel("data/processed_data/ok_1000h.xlsx")

dweather = pd.read_excel("data/processed_data/dvdk_weather.xlsx")
mweather = pd.read_excel("data/processed_data/mesonet.xlsx")

In [ ]:
required_weather_vars = ["Ed", "Ew", "solar", "wind", "rain"]

## Join Weather

The OK Mesonet data is missing air temp for parts of the study period. Joining with air temp data from Van der Kamp.

Then, standardize units and calculate derived predictors

### Handle Missing Observations

Identify areas with spot missing data that can be replaced with interpolation.

In [ ]:
dfw = mweather[["date"] + required_weather_vars]
mask = dfw.isna().any(axis=1)

print(f"Number of Half-Hourly Observations from OK Mesonet: {mweather.shape[0]}")
print(f"Number of Half-Hour Periods with Missing Key Variables: {mask.sum()}")